In [1]:
# Alright lets see what this is all about
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn 
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, plot_roc_curve

In [3]:
#LOAD GIS WATERMAIN AND BREAK DATA
df = pd.read_csv('Kirk_All.csv')
#df.dropna(inplace=True) # Are there any columns where we can deal with NA values?
# Can we use imputation instead?
breaks = pd.read_csv('Kirk_Break_Data.csv')
df['INSTYEAR'] = df['INSTYEAR'].map(lambda x: int((str(x)[-12:-8])), na_action='ignore')
#df = pd.get_dummies(df)
breaks.head()

,Date,Location,Zone,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,SLOPE,Comment,Break_Year,Unnamed: 13
0,1/8,347 7th Ave,315,NaN,NaN,6,CI,NaN,NaN,NaN,NaN,NaN,2004,NaN
1,1/9,696 16th Ave W,315,DIST,NaN,8,AC,NaN,AmC,15000.0,NaN,NaN,2004,NaN
2,1/21,740 16th Ave W,315,DIST,NaN,6,AC,NaN,AmC,15000.0,NaN,NaN,2004,NaN
3,2/25,10132 NE 59th,285,DIST,261.0,8,AC,1976,KpB,7000.0,2.586207,Leak,2004,NaN
4,3/16,18th Ave & 6th St.,450,DIST,NaN,12,AC,1968,AgC,12000.0,NaN,Leak,2004,NaN


In [14]:
df['INSTYEAR'].unique()

array([  nan, 1992., 2005., 1963., 1978., 2002., 2000., 1990., 2007.,
       1988., 1997., 1993., 2009., 2003., 1999., 1987., 1996., 1998.,
       2004., 2006., 1991., 1980., 1989., 2010., 1944., 2001., 2008.,
       1964., 1983., 1968., 1971., 1960., 1970., 1943., 1995., 1975.,
       1961., 1979., 1972., 1953., 1994., 1982., 1976., 1967., 1981.,
       1951., 1985., 1977., 1986., 1969., 1956., 1984., 1949., 1957.,
       1942., 1955., 1965., 1973., 1962., 2020., 1959., 1966., 1974.,
       2012., 2013., 2018., 2011., 2017., 1952., 1958., 1948., 2014.,
       2015., 2016., 2019.])

In [94]:
# EXTRACTING FEATURES
features = ['FID_WA_Mai', 'TYPE', 'LENGTH', 'DIAMETER', 'MATERIAL', 'INSTYEAR', 'MUSYM', 'Dist_Rail', 'SLP_CLASS', 'FULL_STRNA', 'SPEED', 'FUNC_CLASS']
pipes = df[features]
pipes.head()

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,SLP_CLASS,FULL_STRNA,SPEED,FUNC_CLASS
0,1,FIRE,2.0,8.0,UNK,NaN,Ur,6000,2.0,120TH AVE NE,25,MINOR ARTERIAL
1,2,FIRE,2.0,2.0,UNK,1992.0,AgC,7000,1.0,122ND LN NE,25,NEIGHBORHOOD ACCESS
2,3,FIRE,1.0,0.0,UNK,NaN,AgC,6000,0.0,124TH AVE NE,35,PRINCIPAL ARTERIAL
3,4,FIRE,3.0,6.0,DI,2005.0,Ur,4000,0.0,LAKE WASHINGTON BLVD NE,35,PRINCIPAL ARTERIAL
4,5,FIRE,149.0,8.0,AC,1963.0,Ur,7000,6.0,,,


In [95]:

model_feats = [
    'DIAMETER',
    'MATERIAL',
    'INSTYEAR',
    'MUSYM',
    'Dist_Rail',
    'Break_Year'

]
filt_breaks = breaks[model_feats]

filt_breaks.head()


,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,Break_Year
0,6,CI,NaN,NaN,NaN,2004
1,8,AC,NaN,AmC,15000.0,2004
2,6,AC,NaN,AmC,15000.0,2004
3,8,AC,1976,KpB,7000.0,2004
4,12,AC,1968,AgC,12000.0,2004


In [96]:
filt_breaks['Break_Year'].map(lambda x: int(x), na_action='ignore') #Change break_year from string to int
filt_breaks = filt_breaks[filt_breaks['INSTYEAR'] != '1959, 1963'] #Remove mislabeled data
filt_breaks['INSTYEAR'] = filt_breaks['INSTYEAR'].map(lambda x: float(x), na_action='ignore') #Change INSTYEAR from string to float
filt_breaks['AGE'] = filt_breaks['Break_Year'] - filt_breaks['INSTYEAR'] #Calculate new column for age
filt_breaks = filt_breaks.dropna() #Drop NaN values
filt_breaks['BREAK'] = 1 #Add new column for whether pipe broke or not
filt_breaks = filt_breaks.drop("Break_Year", axis=1) #remove break_year column
filt_breaks.head()

,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,AGE,BREAK
3,8,AC,1976.0,KpB,7000.0,28.0,1
4,12,AC,1968.0,AgC,12000.0,36.0,1
5,6,AC,1949.0,AmC,14000.0,55.0,1
6,8,AC,1977.0,AgC,5000.0,27.0,1
7,6,AC,1963.0,AgC,9000.0,41.0,1


In [97]:
# GROUP PIPES BY THEIR UNIQUE ID - NEEDED BECAUSE OF HOW SLOPE BREAKS UP PIPES IN DIFFERENT SLOPE CLASSES
def group_by_id(df):

    grouping = df.groupby('FID_WA_Mai').groups
    unique_pipes = df['FID_WA_Mai'].unique()
    indices = []
    for i in range(len(unique_pipes)):
        pipe = unique_pipes[i]
        val = grouping[pipe][0]
        indices.append(val)
    
    # CALCULATE AVERAGE SLOPES FOR PIPES THAT GOT SPLIT INTO MULTIPLE ROWS
    slopes = df[['FID_WA_Mai', 'SLP_CLASS']]
    slopes = slopes.groupby('FID_WA_Mai')['SLP_CLASS'].mean()
    slopes = pd.Series(slopes)

    filtered = df.loc[indices, 'FID_WA_Mai':'FUNC_CLASS']
    filtered = filtered.drop('SLP_CLASS', axis=1)
    filtered['SLOPE'] = slopes.values

    return filtered

In [98]:
filtered = group_by_id(pipes)
filtered.head()

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
0,1,FIRE,2.0,8.0,UNK,NaN,Ur,6000,120TH AVE NE,25,MINOR ARTERIAL,2.000000
1,2,FIRE,2.0,2.0,UNK,1992.0,AgC,7000,122ND LN NE,25,NEIGHBORHOOD ACCESS,1.000000
2,3,FIRE,1.0,0.0,UNK,NaN,AgC,6000,124TH AVE NE,35,PRINCIPAL ARTERIAL,0.000000
3,4,FIRE,3.0,6.0,DI,2005.0,Ur,4000,LAKE WASHINGTON BLVD NE,35,PRINCIPAL ARTERIAL,0.000000
4,5,FIRE,149.0,8.0,AC,1963.0,Ur,7000,,,,3.888889


In [99]:
more_feats = [
    'DIAMETER',
    'MATERIAL',
    'INSTYEAR',
    'MUSYM',
    'Dist_Rail'
]

filtered_pipes = filtered[more_feats]
filtered_pipes = filtered_pipes.dropna()
filtered_pipes['AGE'] = 2020 - filtered_pipes['INSTYEAR'] #Calculating pipe age
filtered_pipes['BREAK'] = 0 #Making new column for BREAK and setting it to 0
filtered_pipes.head()

,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,AGE,BREAK
1,2.0,UNK,1992.0,AgC,7000,28.0,0
3,6.0,DI,2005.0,Ur,4000,15.0,0
4,8.0,AC,1963.0,Ur,7000,57.0,0
22,12.0,DI,1978.0,Ur,6000,42.0,0
23,6.0,DI,2002.0,AmC,13000,18.0,0


In [107]:
full = pd.concat([filt_breaks, filtered_pipes], axis=0, sort=False) #Combine break and non-break data sets

feats = [
    'DIAMETER',
    'MATERIAL',
    'INSTYEAR',
    'MUSYM',
    'Dist_Rail',
    'AGE'
]


y = full['BREAK'] # Column we want to predict fof the ML model

x = full[feats] # Features for the mode
x = pd.get_dummies(x) # One hot encode categorical data
x.head()

,DIAMETER,INSTYEAR,Dist_Rail,AGE,MATERIAL_AC,MATERIAL_CI,MATERIAL_CONC,MATERIAL_CPR,MATERIAL_Conc cyl,MATERIAL_DI,...,MUSYM_KpB,MUSYM_KpD,MUSYM_Ng,MUSYM_No,MUSYM_PITS,MUSYM_RdC,MUSYM_RdE,MUSYM_So,MUSYM_Tu,MUSYM_Ur
3,8.0,1976.0,7000.0,28.0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,12.0,1968.0,12000.0,36.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6.0,1949.0,14000.0,55.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,8.0,1977.0,5000.0,27.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,6.0,1963.0,9000.0,41.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [115]:
X_train, X_test, y_train, y_test = train_test_split(x, y)
xgb_model = xgb.XGBClassifier()
etc_model = ExtraTreesClassifier()

xgb_model.fit(X_train, y_train)
etc_model.fit(X_train, y_train)

ExtraTreesClassifier()

In [131]:
xgb_preds = xgb_model.predict(X_test)
etc_preds = etc_model.predict(X_test)

#accuracy_score(xgb_preds, y_test)

confusion_matrix(y_test, xgb_preds)

#roc_auc_score(y_test, xgb_preds)

##fpr, tpr, thr = roc_curve(y_test, xgb_preds)

#plt.plot(fpr, tpr)

array([[1726,    0],
       [   1,    2]])

In [204]:
break_1_9_04 = filtered[(filtered['FULL_STRNA'] == '16TH AVE W')] #& (filtered['INSTYEAR'] < 2005) & (filtered['INSTYEAR'] > 2003)]
break_1_9_04 = break_1_9_04[break_1_9_04['INSTYEAR'] == 2004]
break_1_9_04

nearby = filtered[(filtered['FULL_STRNA'] == '16TH AVE W') & (filtered['INSTYEAR'] != 2004) & (filtered['Dist_Rail'] == 15000) & (filtered['DIAMETER'] == 8.0)] 
nearby

similar = filtered[(filtered['DIAMETER'] == 8.0) & (filtered['MATERIAL'] == 'AC') & (filtered['MUSYM'] == 'AmC')]
print(similar['FULL_STRNA'].unique())

similar = similar[(similar['FULL_STRNA'] == '17TH AVE W')]
similar

['10TH AVE' 'KIRKLAND CIR' '4TH ST' '17TH AVE W' '9TH AVE' '5TH AVE'
 '3RD ST' 'KIRKLAND WAY' '8TH LN' '15TH AVE' 'KIRKLAND AVE' 'NE 68TH ST'
 '19TH AVE' '8TH AVE' 'NE 45TH ST' ' ' '1ST ST' 'NE 44TH ST' 'NE 68TH PL'
 '6TH ST' '106TH AVE NE' '6TH AVE' '5TH ST' '7TH AVE' '109TH PL NE'
 '10TH AVE S' '5TH AVE S' 'LAKEVIEW DR' '5TH ST S' '6TH ST S' 'STATE ST S']


,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
3851,709,DIST,34.0,8.0,AC,1968,AmC,14000,17TH AVE W,25,NEIGHBORHOOD ACCESS,0.5


In [205]:
break_1_21_04 = filtered[(filtered['FULL_STRNA'] == '16TH AVE W') & (filtered['INSTYEAR'] == 2004)]
break_1_21_04

nearby = filtered[(filtered['FULL_STRNA'] == '16TH AVE W') & (filtered['INSTYEAR'] != 2004)] #& (filtered['Dist_Rail'] == 15000) & (filtered['DIAMETER'] == 6.0)] 


similar = filtered[(filtered['DIAMETER'] == 8.0) & (filtered['MATERIAL'] == 'AC') & (filtered['MUSYM'] == 'AmC')]
print(similar['FULL_STRNA'].unique())

similar = similar[(similar['FULL_STRNA'] == '17TH AVE W')]
similar

['10TH AVE' 'KIRKLAND CIR' '4TH ST' '17TH AVE W' '9TH AVE' '5TH AVE'
 '3RD ST' 'KIRKLAND WAY' '8TH LN' '15TH AVE' 'KIRKLAND AVE' 'NE 68TH ST'
 '19TH AVE' '8TH AVE' 'NE 45TH ST' ' ' '1ST ST' 'NE 44TH ST' 'NE 68TH PL'
 '6TH ST' '106TH AVE NE' '6TH AVE' '5TH ST' '7TH AVE' '109TH PL NE'
 '10TH AVE S' '5TH AVE S' 'LAKEVIEW DR' '5TH ST S' '6TH ST S' 'STATE ST S']


,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
3851,709,DIST,34.0,8.0,AC,1968,AmC,14000,17TH AVE W,25,NEIGHBORHOOD ACCESS,0.5


In [197]:
break_2_25_04 = filtered[(filtered['FULL_STRNA'] == 'NE 59TH ST')]
break_2_25_04

# THIS WILL LIKELY NEED TO BE altered in data set

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
22011,3634,DIST,1.0,8.0,AC,1976,KpB,7000,NE 59TH ST,25,NEIGHBORHOOD ACCESS,1.000000
22012,3635,DIST,5.0,12.0,AC,1971,KpB,7000,NE 59TH ST,25,NEIGHBORHOOD ACCESS,1.000000
39310,6927,DIST,261.0,8.0,AC,1976,KpB,7000,NE 59TH ST,25,NEIGHBORHOOD ACCESS,2.586207


In [42]:
break_1_04_07 = pipes[(pipes['INSTYEAR'] == 2007) & (pipes['FULL_STRNA'].str.contains('2ND'))]
(break_1_04_07)

nearby = pipes[(pipes['INSTYEAR'] < 2007) & (pipes['DIAMETER'] == 4) & (pipes['MATERIAL'] == 'AC') & (pipes['FULL_STRNA'].str.contains('2ND AVE')) & (pipes['Dist_Rail'] == 12000)]
nearby

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
26578,4438,DIST,197.0,4.0,AC,1991.0,AmC,12000,2ND AVE S,25,NEIGHBORHOOD ACCESS,1.5
40951,7457,DIST,164.0,4.0,AC,1991.0,AmC,12000,2ND AVE S,25,NEIGHBORHOOD ACCESS,3.0
40953,7458,DIST,2.0,4.0,AC,1991.0,AmC,12000,2ND AVE S,25,NEIGHBORHOOD ACCESS,0.0
40954,7459,DIST,3.0,4.0,AC,1991.0,AmC,12000,2ND AVE S,25,NEIGHBORHOOD ACCESS,1.5
40955,7460,DIST,2.0,4.0,AC,1991.0,AmC,12000,2ND AVE S,25,NEIGHBORHOOD ACCESS,0.5


In [52]:
break_10_08_07 = pipes[(pipes['INSTYEAR'] == 2009) & (pipes['FULL_STRNA'].str.contains('9TH AVE'))]
break_10_08_07

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
15914,2593,DIST,4.0,12.0,DI,2009.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,1.0
15915,2594,DIST,4.0,12.0,DI,2009.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,1.0
16042,2606,DIST,4.0,12.0,DI,2009.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,1.0
23601,3893,DIST,4.0,12.0,DI,2009.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,1.0
24474,4073,DIST,4.0,12.0,DI,2009.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,0.8
24507,4078,DIST,59.0,12.0,DI,2009.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,1.0
24510,4079,DIST,415.0,12.0,DI,2009.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,1.0
24527,4081,DIST,9.0,12.0,DI,2009.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,1.0


In [69]:
break_11_2_07 = pipes[(pipes['INSTYEAR'] == 2008) & (pipes['FULL_STRNA'] == '20TH AVE')]
break_11_2_07

nearby = pipes[(pipes['INSTYEAR'] < 2007) & (pipes['FULL_STRNA'] == '20TH AVE')]
nearby

similar = pipes[(pipes['MATERIAL'] == 'AC') & (pipes['DIAMETER'] == 4) & (pipes['MUSYM'] == 'InC') & (pipes['Dist_Rail'] >= 10000)]
nearby

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
9492,1616,DIST,622.0,6.0,AC,1968.0,InC,12000,20TH AVE,25,NEIGHBORHOOD ACCESS,1.058824
9509,1617,DIST,1.0,6.0,AC,1968.0,InC,12000,20TH AVE,25,NEIGHBORHOOD ACCESS,0.000000
10655,1820,DIST,182.0,8.0,DI,1993.0,InC,11000,20TH AVE,25,NEIGHBORHOOD ACCESS,2.125000
10663,1821,DIST,272.0,8.0,DI,1993.0,InC,11000,20TH AVE,25,NEIGHBORHOOD ACCESS,2.800000
10698,1823,DIST,32.0,6.0,AC,1968.0,InC,12000,20TH AVE,25,NEIGHBORHOOD ACCESS,0.000000
15501,2536,DIST,270.0,8.0,DI,2005.0,InC,12000,20TH AVE,25,NEIGHBORHOOD ACCESS,2.600000
35749,6006,DIST,12.0,8.0,DI,2005.0,InC,12000,20TH AVE,25,NEIGHBORHOOD ACCESS,3.000000
35750,6007,DIST,294.0,8.0,DI,1993.0,InC,12000,20TH AVE,25,NEIGHBORHOOD ACCESS,2.000000
36467,6138,DIST,592.0,8.0,DI,1997.0,AmC,13000,20TH AVE,25,NEIGHBORHOOD ACCESS,1.842105
